## Upotrebom niti koje simuliraju *po jednu ćeliju* i sinhronizacijom redovima za poruke  

Svaka nit simulira rad jedne ćelije u sistemu. Stanje svake ćelije se čuva unutar ćelije (rad sistema se ne oslanja na deljenu matricu stanja). Ćelije podatke o svojem stanju razmenjuju putem reda za poruke. Za potrebe analize rada može se uvesti deljeni niz matrica stanja (i-ti element niza je matrica stanja i-te iteracije), u koji ćelije upisuju svoja stanja (ćelije ne mogu čitati iz ovog niza!).


In [1]:
# Imports:
import random, threading, queue, time
import numpy as np

In [2]:
# Global variables:

# Safe to change values:
n_epoch = 50                        # Number of epochs ( iterations ) Conway's game of life have.
n = 30                              # Number of rows and columns ( matrix type: NxN ).

# Not safe to change values:
n_neighbours = 8                    # Number of cell neighbours ( TOP_LEFT, TOP, TOP_RIGHT, LEFT, RIGHT, BOT_LEFT, BOT, BOT_RIGHT ).

# Global structs:
cells = None
steps = list()                      # List of matrixes trough epoches - list[i] is the appereance of the values of the every cell state in the i-th epoch.
queues = list()
write_mutex = threading.Lock()
queue_mutex = threading.Lock()

In [3]:
# Cell

class Cell( threading.Thread ):
    
    def __init__( self, x, y ):
        threading.Thread.__init__( self )

        self.state = random.randint( 0, 1 )                                     # Generating random state for first epoch.
        self.epoch = 0                                                          # Current epoch ( iteration ).

        self.x = x                                                              # The x coordinate of cell in the matrix.
        self.y = y                                                              # The y coordinate of cell in the matrix.
        
        self.neighbour_queue = queue.Queue()


    def __str__( self ):
           return f'[ { self.x } ][ { self.y } ] - { self.state }'


    def notify_neighbours_queues( self ):

        x = self.x
        y = self.y  
        state = self.state

        cells[ ( x - 1 ) % n ][ ( y - 1 ) % n ].neighbour_queue.put( state )
        cells[ ( x - 1 ) % n ][ y ].neighbour_queue.put( state )
        cells[ ( x - 1 ) % n ][ ( y + 1 ) % n ].neighbour_queue.put( state )
        cells[ x ][ ( y - 1 ) % n ].neighbour_queue.put( state )
        cells[ x ][ ( y + 1 ) % n ].neighbour_queue.put( state )
        cells[ ( x + 1 ) % n ][ ( y - 1 ) % n ].neighbour_queue.put( state )
        cells[ ( x + 1 ) % n ][ y ].neighbour_queue.put( state )
        cells[ ( x + 1 ) % n ][ ( y + 1 ) % n ].neighbour_queue.put( state )

    def run( self ):
        
        global cells, steps, cells_left, is_saved, cell_queue

        # Every thread has to loop n_epoch times.
        for epoch in range( 0, n_epoch ):
            
            queue_mutex.acquire()

            try:
                queues[epoch].put( self )
            except IndexError:
                queues.append( queue.Queue() )
                queues[epoch].put( self )

            queue_mutex.release()
            

            self.notify_neighbours_queues()
        
            alive = 0

            for _ in range ( 0, n_neighbours ):
                try:
                    alive += self.neighbour_queue.get()
                except queue.Empty:
                    pass

            # Cell is alive. #
            if self.state == 1:
                if alive < 2 or alive > 3:
                    self.state = 0
            
            # Cell is dead.
            else:
                if alive == 3:
                    self.state = 1


            write_mutex.acquire() 

            try:
                steps[epoch]
            except IndexError:
                steps.append( np.array( [ [ cells[i][j].state for j in range( n ) ] for i in range( n ) ] ) )
            
            write_mutex.release()

            queues[epoch].get()
            queues[epoch].task_done()          # Used to support queue.Queue.join().
            queues[epoch].join()

            self.epoch = epoch     

In [4]:
# Main

def main():

    global cells, steps, cells_left, x

    # Generating cells with random state.
    cells = [ [ Cell( i, j ) for j in range( n ) ] for i in range( n ) ]

    steps = list()

    # Generating initial step matrix with cell states.
    steps.append( np.array( [ [ cells[i][j].state for j in range( n ) ] for i in range( n ) ] ) )
    
    for i in range( n ):
        for j in range( n ):
            cells[i][j].start()

    for i in range( n ):
        for j in range( n ):
            cells[i][j].join()
    
if __name__ == "__main__":
    main()

In [ ]:
from matplotlib.animation import FuncAnimation
import matplotlib.pyplot as plt
from IPython.display import HTML

def animate( steps ):
  '''
  Animates the array of matrices - each matrix is a single state in simulation.
  '''
  
  def init():
    im.set_data( steps[0] )
    return [ im ]
  
  
  def animate( i ):
    im.set_data( steps[i] )
    return [ im ]


  im = plt.matshow( steps[0], interpolation = 'None', animated = True );
  
  anim = FuncAnimation( im.get_figure(), animate, init_func = init,
                  frames = len( steps ), interval = 1000, blit = True, repeat = False );
                  
  return anim

anim = animate( steps );
HTML( anim.to_html5_video() )